In [34]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.callbacks import get_openai_callback
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.embeddings import OpenAIEmbeddings

In [73]:
import os
os.environ["LANGCHAIN_TRACING"] = "true"
os.environ["LANGCHAIN_SESSION"] = "pdf_query"
os.environ["OPENAI_API_KEY"] = "api key"

In [3]:
with get_openai_callback() as cb:
    pdf_folder_path = "data/"
    loaders = [UnstructuredPDFLoader(os.path.join(pdf_folder_path, fn)) for fn in os.listdir(pdf_folder_path)]
    print(cb)

Tokens Used: 0
	Prompt Tokens: 0
	Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


In [26]:
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)

In [64]:
persist_directory = 'db'
for document in loaders:
    data = document.load()
    texts = text_splitter.split_documents(data)
    embeddings = OpenAIEmbeddings()
    
    vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embeddings,
                                 persist_directory=persist_directory)
    vectordb.persist()
    vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embeddings)
    print(db)

In [65]:
retriever = vectordb.as_retriever()

In [66]:
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [67]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [70]:
with get_openai_callback() as cb:
    query = "applications of artificial inteligence"
    llm_response = qa_chain(query)
    process_llm_response(llm_response)
    print(cb)

Error in LangChainTracerV1.on_chain_end callback: Unknown run type: retriever


 Some applications of artificial intelligence include voice recognition, virtual agents, machine learning platforms, AI optimized hardware, decision management, deep learning platforms, biomatters, robotic process automation, text analytics and NLP, and adaptive manufacturing.


Sources:
data/ArtificialIntellegence.pdf
data/ArtificialIntellegence.pdf
data/ArtificialIntellegence.pdf
data/ArtificialIntellegence.pdf
Tokens Used: 1126
	Prompt Tokens: 1082
	Completion Tokens: 44
Successful Requests: 1
Total Cost (USD): $0.022520000000000002
